In [1]:
%pip install gym-super-mario-bros

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -r requirements.txt 

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install 'stable-baselines3[extra]'

Note: you may need to restart the kernel to use updated packages.


In [5]:
from nes_py.wrappers import JoypadSpace
import gym
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT
env = gym.make('SuperMarioBros-v0')
env = JoypadSpace(env, COMPLEX_MOVEMENT)

/home/aayush_ad/Code/.conda/lib/python3.8/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


In [6]:
done = True
env.reset()
for step in range(500):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    # env.render()
    if done:
       state = env.reset()

In [7]:
import torch
torch.cuda.is_available()

True

In [8]:
from stable_baselines3 import PPO

In [9]:
model = PPO("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=7_000)

vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/aayush_ad/Code/.conda/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 116  |
|    iterations      | 1    |
|    time_elapsed    | 17   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 76          |
|    iterations           | 2           |
|    time_elapsed         | 53          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.019369502 |
|    clip_fraction        | 0.322       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.47       |
|    explained_variance   | -0.000687   |
|    learning_rate        | 0.0003      |
|    loss                 | 2.17        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0179     |
|    value_loss           | 20.4        |
-----------------------------------------
----------------------------------